In [ ]:
# Load systems
%run warhammer_ai_main.ipynb
%run expanded_units.ipynb

import pandas as pd
import seaborn as sns

class StrategicAnalyzer:
    """Tool for analyzing army effectiveness and matchups"""
    
    def __init__(self):
        self.nuln_units = expanded_nuln
        self.enemy_units = expanded_enemies
        self.matchup_matrix = {}
        
    def analyze_army_composition(self, composition):
        """Analyze an army's tactical profile"""
        analysis = {
            'total_points': 0,
            'total_models': 0,
            'ranged_power': 0,
            'melee_power': 0,
            'artillery_count': 0,
            'mobility': 0,
            'durability': 0,
            'leadership': 0,
            'unit_breakdown': {}
        }
        
        for unit_key, size in composition.items():
            if unit_key in self.nuln_units:
                unit = self.nuln_units[unit_key]
                
                # Calculate costs and counts
                points = unit.points_cost * size if unit.unit_type != UnitType.CHARACTER else unit.points_cost
                analysis['total_points'] += points
                analysis['total_models'] += size
                
                # Analyze capabilities
                has_ranged = any(eq.weapon_type == WeaponType.RANGED for eq in unit.equipment)
                has_artillery = any(eq.weapon_type == WeaponType.ARTILLERY for eq in unit.equipment)
                
                if has_artillery:
                    analysis['artillery_count'] += 1
                    analysis['ranged_power'] += unit.strength * 2  # Artillery is powerful
                elif has_ranged:
                    analysis['ranged_power'] += unit.ballistic_skill * size
                
                analysis['melee_power'] += unit.weapon_skill * unit.attacks * size
                analysis['mobility'] += unit.movement * size
                analysis['durability'] += unit.toughness * unit.wounds * size
                analysis['leadership'] += unit.leadership * size
                
                analysis['unit_breakdown'][unit.name] = {
                    'models': size,
                    'points': points,
                    'role': 'Artillery' if has_artillery else 'Ranged' if has_ranged else 'Melee'
                }
        
        # Normalize metrics
        if analysis['total_models'] > 0:
            analysis['avg_mobility'] = analysis['mobility'] / analysis['total_models']
            analysis['avg_durability'] = analysis['durability'] / analysis['total_models']
            analysis['avg_leadership'] = analysis['leadership'] / analysis['total_models']
        
        return analysis
    
    def run_matchup_analysis(self, nuln_armies, enemy_types, battles_per_matchup=10):
        """Run comprehensive matchup analysis"""
        results = []
        
        print("🔍 Running Strategic Matchup Analysis...")
        print(f"Testing {len(nuln_armies)} Nuln armies vs {len(enemy_types)} enemy types")
        print(f"Battles per matchup: {battles_per_matchup}")
        print("-" * 50)
        
        for army_name, nuln_comp in nuln_armies.items():
            for enemy_name, enemy_comp in enemy_types.items():
                print(f"Testing {army_name} vs {enemy_name}...")
                
                # Run battles
                wins = 0
                for battle in range(battles_per_matchup):
                    winner = self._simulate_battle(nuln_comp, enemy_comp)
                    if winner == 1:
                        wins += 1
                
                win_rate = wins / battles_per_matchup
                
                # Analyze compositions
                nuln_analysis = self.analyze_army_composition(nuln_comp)
                
                results.append({
                    'nuln_army': army_name,
                    'enemy_type': enemy_name,
                    'win_rate': win_rate,
                    'nuln_points': nuln_analysis['total_points'],
                    'nuln_models': nuln_analysis['total_models'],
                    'ranged_power': nuln_analysis['ranged_power'],
                    'melee_power': nuln_analysis['melee_power'],
                    'artillery_count': nuln_analysis['artillery_count'],
                    'avg_mobility': nuln_analysis.get('avg_mobility', 0),
                    'avg_durability': nuln_analysis.get('avg_durability', 0)
                })
        
        return pd.DataFrame(results)
    
    def _simulate_battle(self, nuln_comp, enemy_comp):
        """Simulate a single battle"""
        # Build armies
        nuln_army = []
        for unit_key, size in nuln_comp.items():
            if unit_key in self.nuln_units:
                unit = copy.deepcopy(self.nuln_units[unit_key])
                if unit.unit_type != UnitType.CHARACTER:
                    unit.models_count = size
                    unit.current_models = size
                unit.position = (random.randint(5, 15), random.randint(5, 15))
                nuln_army.append(unit)
        
        enemy_army = []
        for unit_key, size in enemy_comp.items():
            if unit_key in self.enemy_units:
                unit = copy.deepcopy(self.enemy_units[unit_key])
                if unit.unit_type != UnitType.CHARACTER:
                    unit.models_count = size
                    unit.current_models = size
                unit.position = (random.randint(30, 40), random.randint(30, 40))
                enemy_army.append(unit)
        
        # Create game
        board = Board()
        game_state = GameState(
            board=board,
            player1_army=nuln_army,
            player2_army=enemy_army
        )
        
        # Play with AI agents
        mcts_agent = MCTSAgent(iterations=25)  # Fast simulation
        heuristic_agent = HeuristicAgent()
        
        ga = GeneticAlgorithm()
        return ga.play_game(game_state, mcts_agent, heuristic_agent, max_turns=30)
    
    def create_army_presets(self):
        """Create preset army compositions for testing"""
        presets = {
            'Gunline': {
                'nuln_state_troops_handgun': 10,
                'nuln_handgunners': 10,
                'nuln_great_cannon': 1,
                'nuln_engineer': 1
            },
            'Artillery Heavy': {
                'nuln_crossbowmen': 10,
                'nuln_great_cannon': 1,
                'nuln_mortar': 1,
                'master_engineer': 1
            },
            'Balanced': {
                'nuln_state_troops_halberd': 10,
                'nuln_crossbowmen': 10,
                'nuln_outriders': 5,
                'nuln_captain': 1
            },
            'Elite Strike': {
                'nuln_handgunners': 15,
                'nuln_outriders': 8,
                'nuln_steam_tank': 1
            },
            'Militia Horde': {
                'nuln_militia': 30,
                'nuln_state_troops_handgun': 10,
                'nuln_great_cannon': 1
            }
        }
        
        enemy_presets = {
            'Orc Horde': {
                'orc_warriors': 15,
                'orc_warboss': 1
            },
            'Chaos Elite': {
                'chaos_warriors': 12
            },
            'Skaven Swarm': {
                'skaven_clanrats': 25
            },
            'Empire Knights': {
                'empire_knights': 6
            },
            'Undead Legion': {
                'skeleton_warriors': 20
            }
        }
        
        return presets, enemy_presets

# Create analyzer
analyzer = StrategicAnalyzer()
nuln_presets, enemy_presets = analyzer.create_army_presets()

print("🎯 STRATEGIC ANALYZER READY!")
print(f"📋 Nuln army presets: {list(nuln_presets.keys())}")
print(f"🗡️ Enemy presets: {list(enemy_presets.keys())}")
print("\\nUse analyzer.run_matchup_analysis() to test all combinations!")
